# 0.0. Imports

In [1]:
import sys
import joblib 

sys.path.insert(0, '../src/')

from sklearn.pipeline import Pipeline
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import StandardScaler

from data.data_load import DataLoad
from data.data_validation import DataValidation
from data.data_transformation import DataTransformation
from utils.utils import load_config_file
from data.data_preprocess import DataPreprocess
from train.train import TrainModels
from sklearn.linear_model import LogisticRegression
from evaluation.classifier_eval import ModelEvaluation

# 1.0. Data load

In [2]:
dl = DataLoad()
df = dl.load_data('train_dataset_name')

2023-12-04 20:09:29 [info     ] Comecando a carga dos dados com o nome: train_dataset_name


# 2.0. Data validation

In [3]:
dv = DataValidation()
is_valid = dv.run(df)

2023-12-04 20:09:30 [info     ] Validacao iniciou
2023-12-04 20:09:30 [info     ] Validation columns passed...
2023-12-04 20:09:30 [info     ] Validacao com sucesso.


# 3.0. Data transformation

In [4]:
dt = DataTransformation(df)
X_train, X_valid, y_train, y_valid = dt.train_test_spliting()

# 4.0. Experimentations

In [5]:
import mlflow 
from mlflow.tracking import MlflowClient

/home/mathdeoliveira/repos/disciplina_mlflow/.env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mathdeoliveira/repos/disciplina_mlflow/.env/lib/python3.10/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [6]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('prob_loan')

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1701729825978, experiment_id='1', last_update_time=1701729825978, lifecycle_stage='active', name='prob_loan', tags={}>

## 4.1. Hyperparameters

In [8]:
from mlflow.models import MetricThreshold
from sklearn.dummy import DummyClassifier
from mlflow.models import infer_signature

In [9]:
from hyperopt import fmin, tpe, hp, STATUS_OK

In [12]:
pipe = Pipeline([('imputer', MeanMedianImputer(variables=load_config_file().get('vars_imputer'))),
                 ('discretizer', EqualFrequencyDiscretiser(variables=load_config_file().get('vars_discretizer'))),
                 ('scaler', SklearnTransformerWrapper(StandardScaler()))])

In [14]:
def objective(params):
    with mlflow.start_run(run_name='with_discretizer_hyperopt'):
        mlflow.set_tag('model_name', 'lr_hyperopt')
        mlflow.log_params(params)
        
    
        preprocessador = DataPreprocess(pipe)
        preprocessador.train(X_train)
        
        X_train_processed = preprocessador.transform(X_train)
        X_valid_processed = preprocessador.transform(X_valid)
        joblib.dump(preprocessador, '../models/preprocess.joblib')
        
        ## 1.1. logar um artefato (preprocessador)
        mlflow.log_artifact('../models/preprocess.joblib')
        
        ## 1.2. logar os parametros do preprocessador
        mlflow.log_params(params={'imputer': pipe['imputer'],
                                'discretizer': pipe['discretizer'],
                                'scaler': pipe['scaler']})
        
        # 2.0. inicia o experimento com cross validatin
        model = LogisticRegression(**params)
        model_eval = ModelEvaluation(model,
                                    X_train_processed,
                                    y_train,
                                    n_splits=5)
        roc_auc_scores = model_eval.cross_val_evaluate()
        ## 2.1. logar o resultado da performance
        mlflow.log_metric('train_roc_auc', roc_auc_scores.mean())
        
        # 3.0. treinar o modelo
        model.fit(X_train_processed, y_train)
        
        # 4.0. salvar as metricas de validacao do modelo
        y_val_preds = model_eval.model.predict_proba(X_valid_processed)[:, 1]
        val_roc_auc = model_eval.evaluate_predictions(y_valid, y_val_preds)
        
        mlflow.log_metric('valid_roc_auc', val_roc_auc)
        
        # 5.0. logar o modelo
        candidate_model_uri = mlflow.sklearn.log_model(model,
                                'lr_model').model_uri
        
        ########
        signature = infer_signature(X_valid_processed, y_valid)
        
        eval_data = X_valid_processed
        eval_data['label'] = y_valid
        
        thresholds = {
            "accuracy_score": MetricThreshold(
                threshold=0.1, # o score da acuracia precisa ser >0.7
                min_absolute_change=0.05,
                min_relative_change=0.05, 
                greater_is_better=True
            )
        }
        
        baseline_model = DummyClassifier(strategy='uniform').fit(X_train_processed, y_train)
        baseline_model_uri = mlflow.sklearn.log_model(baseline_model,
                                                    "baseline_model",
                                                    signature=signature).model_uri
        
        # processo responsavel por avaliar o modelo do mlflow
        mlflow.evaluate(candidate_model_uri,
                        eval_data,
                        targets="label",
                        model_type="classifier",
                        validation_thresholds=thresholds,
                        baseline_model=baseline_model_uri)
        
        
        mlflow.end_run()
        
        return {'loss': -roc_auc_scores.mean(),
                'status': STATUS_OK}

In [11]:
search_space = {'warm_start' : hp.choice('warm_start', [True, False]),
                'fit_intercept' : hp.choice('fit_intercept', [True, False]),
                'tol' : hp.uniform('tol', 0.00001, 0.0001),
                'C' : hp.uniform('C', 0.05, 3),
                'solver' : hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
                'max_iter' : hp.choice('max_iter', range(100,1000)),
                'multi_class' : 'auto',
                'class_weight' : hp.choice('class_weight', [None, 'balanced'])}

In [15]:
best_result = fmin(fn=objective,
                   space=search_space,
                   algo=tpe.suggest,
                   max_evals=5)

2023-12-04 20:16:38 [info     ] Preprocessamento iniciou...
2023-12-04 20:16:38 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:16:38 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:16:38 [info     ] Iniciou a validacao cruzada...
2023-12-04 20:16:39 [info     ] Iniciou a validacao do modelo
  0%|          | 0/5 [00:01<?, ?trial/s, best loss=?]

Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.

Setuptools is replacing distutils.

Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/doc

2023-12-04 20:16:46 [info     ] Preprocessamento iniciou...
2023-12-04 20:16:46 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:16:46 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:16:46 [info     ] Iniciou a validacao cruzada...
2023-12-04 20:16:49 [info     ] Iniciou a validacao do modelo
 20%|██        | 1/5 [00:11<00:32,  8.08s/trial, best loss: -0.8066863323994398]

The line search algorithm did not converge

Line Search failed

Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.

Setuptools is replacing distutils.

Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `H

2023-12-04 20:16:56 [info     ] Preprocessamento iniciou...
2023-12-04 20:16:56 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:16:56 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:16:56 [info     ] Iniciou a validacao cruzada...
2023-12-04 20:16:58 [info     ] Iniciou a validacao do modelo
 40%|████      | 2/5 [00:19<00:27,  9.29s/trial, best loss: -0.8066863323994398]

...

2023-12-04 20:17:04 [info     ] Preprocessamento iniciou...
2023-12-04 20:17:05 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:17:05 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:17:05 [info     ] Iniciou a validacao cruzada...
2023-12-04 20:17:05 [info     ] Iniciou a validacao do modelo
 60%|██████    | 3/5 [00:27<00:17,  8.77s/trial, best loss: -0.8066863323994398]

Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.

Setuptools is replacing distutils.

Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/doc

2023-12-04 20:17:12 [info     ] Preprocessamento iniciou...
2023-12-04 20:17:12 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:17:12 [info     ] Transformacao dos dados com preprocessador iniciou...
2023-12-04 20:17:13 [info     ] Iniciou a validacao cruzada...
2023-12-04 20:17:15 [info     ] Iniciou a validacao do modelo
 80%|████████  | 4/5 [00:37<00:08,  8.45s/trial, best loss: -0.8066863323994398]

Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.

Setuptools is replacing distutils.

Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/doc

100%|██████████| 5/5 [00:43<00:00,  8.74s/trial, best loss: -0.8066863323994398]
